In [1]:
import math
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def apply_vectorized(functions: list[callable], array: np.array):
    result = np.zeros(array.shape[0], dtype=float)
    for i, f in enumerate(functions):
        result[i] = f(array[i])
    return result

In [3]:
apply_vectorized = np.vectorize(lambda f, x: f(x), otypes=[object])

given 3 neurons, defs:

In [8]:
x = lambda t: 1 / t
weights = np.array([[-0.2,  -1, -0.8], 
                    [   1, 0.8, -1.5], 
                    [ 0.5, 0.6, -0.4]], dtype=float)
initial_weigths = np.array([0.5, 0, -0.5], dtype=float)
activation_functions = [
    lambda h: math.tanh(h),
    lambda h: math.tanh(h),
    lambda h: math.exp(-h*h/2)
    ]
Y = np.array([0.1, 0.1, 0.1], dtype=float)
T = 100


In [165]:
def syncPropagation(weights, initial_weigths, activation_functions, x, t, Y):
    return apply_vectorized(activation_functions, weights.dot(Y) + initial_weigths * x(t))

In [5]:
def syncPropagation(weights, initial_weigths, activation_functions, x, t, Y):
    nextY = Y.copy()
    # h = np.zeros((3))
    for i in range(weights.shape[0]):
        h = 0
        for j in range(weights[i].shape[0]):
            h += Y[j] * weights[i][j]
        h += initial_weigths[i]*x(t)
        nextY[i] = activation_functions[i](h)
    # nextY = apply_vectorized(activation_functions, h)
    return nextY

In [6]:
def asyncPropagation(weights, initial_weigths, activation_functions, x, t, Y):
    nextY = Y.copy()
    # h = np.zeros((3))
    for i in range(weights.shape[0]):
        h = 0
        for j in range(weights[i].shape[0]):
            h += nextY[j] * weights[i][j]
        h += initial_weigths[i]*x(t)
        nextY[i] = activation_functions[i](h)
    # nextY = apply_vectorized(activation_functions, h)
    return nextY

In [9]:
Ys = {
    'sync': np.zeros((T, 3)),
    'async': np.zeros((T, 3))   
}

ts = np.linspace(0, T, T)
for i, t in enumerate(ts):
    if i == 0:
        Ys['sync'][i] = Y.copy()
        Ys['async'][i] = Y.copy()
    else:
        Ys['sync'][i] = syncPropagation(weights, initial_weigths, activation_functions, x, t, Ys['sync'][i - 1])
        Ys['async'][i] = asyncPropagation(weights, initial_weigths, activation_functions, x, t, Ys['async'][i - 1])

Yst = {
    'sync': Ys['sync'].transpose(),
    'async': Ys['async'].transpose() 
}

fig = go.Figure()
for i in range(Yst['sync'].shape[0]):
    fig.add_trace(go.Scatter(x = ts, y = Yst['sync'][i], name = f'Sync Neuron {i+1}'))
    fig.add_trace(go.Scatter(x = ts, y = Yst['async'][i], name = f'Async Neuron {i+1}'))
fig.show()


In [148]:
Ys = np.zeros((T, 3))

ts = np.linspace(0, T, T)
for i, t in enumerate(ts):
    if i == 0:
        Ys[i] = Y.copy()
    else:
        Ys[i] = asyncPropagation(weights, initial_weigths, activation_functions, x, t, Ys[i - 1])

Yst = Ys.transpose()
fig = go.Figure()
for i in range(Ys.shape[1]):
    fig.add_trace(go.Scatter(x = ts, y = Yst[i], name = f'Neuron {i+1}'))
fig.show()